# Run cells below to install required libraries

In [ ]:
import sys
!{sys.executable} -m pip install pymysql
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install plotly

# Fill in the variables in the cell below

In [ ]:
DATABASEUSER = 'squadjs'
DATABASEPSWD = 'password'
DATABASEHOST = '127.0.0.1'
DATABASENAME = 'squadjs'
DATABASEPORT = 3306 # default port, may be different depending on your setup

SERVERNUMBER_UE5 = 2 
SERVERNUMBER_UE4 = 1

TIMEZONE = "UTC" # Depends on servertime, not local time without daylight savings zones (no EDT, just EST)

# Don't touch below, just run the cells

In [ ]:
import pymysql
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo
import plotly.graph_objects as go

In [ ]:
def connect_database() -> pymysql.connections.Connection:
    connection = pymysql.connect(host=DATABASEHOST, user = DATABASEUSER, password= DATABASEPSWD, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor, database=DATABASENAME, port=DATABASEPORT)
    return connection


In [ ]:
def generate_matches_list(serverNum):
    
    utcStartTime = datetime(2025, 5, 30, 18, 0, 0, tzinfo=ZoneInfo("UTC"))
    startTimeStr = utcStartTime.astimezone(ZoneInfo(TIMEZONE)).strftime('%Y-%m-%d %H:%M:%S')
    utcEndTime = datetime(2025, 6, 2, 18, 0, 0, tzinfo=ZoneInfo("UTC"))
    endTimeStr = utcEndTime.astimezone(ZoneInfo(TIMEZONE)).strftime('%Y-%m-%d %H:%M:%S')

    with connect_database() as connection:
        with connection.cursor() as cursor:
            sql = f'SELECT id FROM squadjs.DBLog_Matches WHERE squadjs.DBLog_Matches.server = {serverNum} AND startTime > "{startTimeStr}" AND startTime < "{endTimeStr}";'
            cursor.execute(sql)
            results = cursor.fetchall()
    matches = []
    for v in results:
        matches.append(v['id'])
    return matches

In [ ]:
def generateGraphDf(matches):
    dfs = []
    for match in matches:
        sql = "SELECT * FROM squadjs.DBLog_TickRates WHERE squadjs.DBLog_TickRates.match = %s"
        with connect_database() as connection:
            with connection.cursor() as cursor:
                cursor.execute(sql, match)
                result = cursor.fetchall()
                if (pd.DataFrame(result).shape[0] > 180 and pd.DataFrame(result).shape[0] < 720):
                    dfs.append(result)
    dfs = sorted(dfs, key= lambda df: pd.DataFrame(df).shape[0], reverse=True)

    new_dfs = []
    for df in dfs:
        df = pd.DataFrame(df)
        startTime = df['time'][0]
        df['time'] = df.apply(lambda row : (row['time'] - startTime).total_seconds(), axis=1)
        df.drop(df.tail(18).index, inplace=True)
        new_dfs.append(df)
    main_df = pd.DataFrame()
    for i in range(len(dfs)):
        df = new_dfs[i]
        match = matches[i]
        colName = f'TPS_Match_{match}'
        #colNames.append(colName)
        
        main_df[colName] = df['tickRate']

    main_df['totalTPS'] = main_df.sum(axis=1)
    main_df['MissingValueCount'] = main_df.isna().sum(axis=1)
    main_df['avgTPS'] = main_df.apply(lambda row: row['totalTPS']/(len(dfs) - row['MissingValueCount']), axis=1)
    main_df['time'] = main_df.index
    main_df.time = main_df.apply(lambda row : row.time/6, axis=1)

    return main_df    

In [ ]:


matchesUE5 = generate_matches_list(SERVERNUMBER_UE5)
matchesUE4 = generate_matches_list(SERVERNUMBER_UE4)

UE5DF = generateGraphDf(matchesUE5)
UE4DF = generateGraphDf(matchesUE4)

main_df = pd.DataFrame()

UE4DF = generateGraphDf(matchesUE4)
if (UE5DF.shape[0]> UE4DF.shape[0]):
    main_df['time'] = UE5DF.time
    main_df['UE5_TPS'] = UE5DF.avgTPS
    main_df['UE4_TPS'] = UE4DF.avgTPS
else:
    main_df['time'] = UE4DF.time
    main_df['UE4_TPS'] = UE4DF.avgTPS
    main_df['UE5_TPS'] = UE5DF.avgTPS

In [ ]:
fig = go.Figure(layout=dict(
    title=dict(text = 'Average TPS during a match',
               subtitle = dict(text = 'Graph generated by [DoD] Leon')
               ),
    xaxis=dict(title = dict( text = 'Time (Minutes)')),
    yaxis=dict(title = dict( text = 'TPS (Ticks per Second)')),
    legend = dict(title = dict(text = 'Version of Unreal Engine'))
))
fig.add_trace(go.Scatter(x= main_df.time, y= main_df.UE4_TPS, name = 'UE4'))
fig.add_trace(go.Scatter(x= main_df.time, y= main_df.UE5_TPS, name = 'UE5'))
fig.update_xaxes(range = [0, 120])
fig.update_yaxes(range = [0, 70])
fig.show()

fig.write_html('avgTPSGraph.html')